In [16]:
import py2neo

airport_names = ["Dublin", "Paris","Toulouse", "Bordeaux", "London", "Malaga"]
airport_countries = ["Ireland","France","France","France","Britain","Spain"]

airport_dict = {}

for name, country in zip(airport_names, airport_countries):
    l = airport_dict.get(country, [])
    l.append(name)
    airport_dict[country] = l
    
print airport_dict

print "CREATE", ",".join ( [ '({name}:Airport {{name:"{name}"}})'.format(name=a) for a in airport_names])

{'Britain': ['London'], 'Spain': ['Malaga'], 'Ireland': ['Dublin'], 'France': ['Paris', 'Toulouse', 'Bordeaux']}
CREATE (Dublin:Airport {name:"Dublin"}),(Paris:Airport {name:"Paris"}),(Toulouse:Airport {name:"Toulouse"}),(Bordeaux:Airport {name:"Bordeaux"}),(London:Airport {name:"London"}),(Malaga:Airport {name:"Malaga"})


In [17]:
from py2neo import authenticate, Graph, Relationship
g = Graph("http://localhost:7474/db/data/")


In [18]:
g.delete_all()
from py2neo import Node
try:
    g.schema.create_uniqueness_constraint("Airport", "name")
except:
    pass

import itertools

for country, airports in airport_dict.iteritems():
    nodes = [Node("Airport", name=a) for a in airports]
    for node in nodes:
        g.create( node )
    
    if len(nodes) > 1:
        same_country = list(itertools.combinations ( nodes , 2 ))
        print same_country, "list"
        for a,b in same_country:
            print "ffff",a,b
            g.create(Relationship(a, "Same_Country", b))

[((paris:Airport {name:"Paris"}), (toulouse:Airport {name:"Toulouse"})), ((paris:Airport {name:"Paris"}), (bordeaux:Airport {name:"Bordeaux"})), ((toulouse:Airport {name:"Toulouse"}), (bordeaux:Airport {name:"Bordeaux"}))] list
ffff (paris:Airport {name:"Paris"}) (toulouse:Airport {name:"Toulouse"})
ffff (paris:Airport {name:"Paris"}) (bordeaux:Airport {name:"Bordeaux"})
ffff (toulouse:Airport {name:"Toulouse"}) (bordeaux:Airport {name:"Bordeaux"})


In [19]:
airports = list(g.find("Airport"))

In [20]:
import random
potential_flights = list ( itertools.permutations ( airports, 2) )
r = random.Random(117)

for i in range(26):
    to, dest = r.choice(potential_flights)
    print to, dest
    g.create(Relationship(to, "Flies_To", dest, cost = random.randint(20,100)))

(malaga:Airport {name:"Malaga"}) (paris:Airport {name:"Paris"})
(london:Airport {name:"London"}) (bordeaux:Airport {name:"Bordeaux"})
(dublin:Airport {name:"Dublin"}) (malaga:Airport {name:"Malaga"})
(london:Airport {name:"London"}) (bordeaux:Airport {name:"Bordeaux"})
(dublin:Airport {name:"Dublin"}) (paris:Airport {name:"Paris"})
(london:Airport {name:"London"}) (toulouse:Airport {name:"Toulouse"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (malaga:Airport {name:"Malaga"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(toulouse:Airport {name:"Toulouse"}) (bordeaux:Airport {name:"Bordeaux"})
(london:Airport {name:"London"}) (malaga:Airport {name:"Malaga"})
(bordeaux:Airport {name:"Bordeaux"}) (toulouse:Airport {name:"Toulouse"})
(dublin:Airport {name:"Dublin"}) (toulouse:Airport {name:"Toulouse"})
(london:Airport {name:"London"}) (bordeaux:Airport {name:"Bordeaux"})
(dublin:Airport {name:"Dublin"}) (malaga:Airport

# Modelling Question
Should the Relationship Same_Country connect flights or airorts? My original thought was Airport as you can see below.
However I think if you are looking at following a path through a graph then you may want to connect the flights.  
  
When you add a flight to the graph you might have to develop routines for searching the graph to add relationships. So a new flight is added from Paris to Cork. For same country relationship should all later flights from Ireland to France have this as a potential connector?  
  
Or is this something that should be done in the front end.

In [21]:
fly_from_same_country = """
MATCH (malaga:Airport) - [f1:Flies_To] -> (a:Airport) -[s:Same_Country] -(b:Airport) - [f2:Flies_To]-> (dublin:Airport)
WHERE malaga.name='Malaga' AND dublin.name = "Dublin"
RETURN  *
ORDER BY f1.cost + f2.cost ASC
"""
results = list(g.run(fly_from_same_country))
print "RES", len(results)
for r in results:
    print r['f1'], r['f2']

RES 1
(malaga)-[:Flies_To {cost:72}]->(paris) (bordeaux)-[:Flies_To {cost:70}]->(dublin)
